In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# 载入数据
mnist = input_data.read_data_sets("MNIST_data", one_hot=True) # 下载数据

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# 批次大小
batch_size = 100
number_neural = 400
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size # // 整除

# 定义两个 placeholder
x = tf.placeholder(tf.float32, [None, 784]) # None 表示任意维度
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)


# 创建一个简单的神经网络
W_1 = tf.Variable(tf.truncated_normal([784,number_neural], stddev=0.1)) # 截断的正太分布
b_1 = tf.Variable(tf.zeros([number_neural])+0.1) # 初始化偏置值为0.1
L_1 = tf.nn.tanh(tf.matmul(x, W_1) + b_1)
L_1_dropout = tf.nn.dropout(L_1, keep_prob) # keep_prob = 0.8 80% 的神经元工作

W_2 = tf.Variable(tf.truncated_normal([number_neural,number_neural], stddev=0.1)) # 截断的正太分布
b_2 = tf.Variable(tf.zeros([number_neural])+0.1) # 初始化偏置值为0.1
L_2 = tf.nn.tanh(tf.matmul(L_1_dropout, W_2) + b_2)
L_2_dropout = tf.nn.dropout(L_2, keep_prob) # keep_prob = 0.8 80% 的神经元工作

W_3 = tf.Variable(tf.truncated_normal([number_neural,number_neural], stddev=0.1)) # 截断的正太分布
b_3 = tf.Variable(tf.zeros([number_neural])+0.1) # 初始化偏置值为0.1
L_3 = tf.nn.tanh(tf.matmul(L_2_dropout, W_3) + b_3)
L_3_dropout = tf.nn.dropout(L_3, keep_prob) # keep_prob = 0.8 80% 的神经元工作

# 输出层
W_4 = tf.Variable(tf.truncated_normal([number_neural,10], stddev=0.1)) # 截断的正太分布
b_4 = tf.Variable(tf.zeros([10])+0.1) # 初始化偏置值为0.1

predict = tf.nn.softmax(tf.matmul(L_3_dropout,W_4)+b_4)

# softmax
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=predict))
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 结果存放在一个bool 型列表中
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(predict,1)) # tf.equal 
# 求准确率
arruracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(31):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size=batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.0})
        
        train_acc = sess.run(arruracy, feed_dict={x:mnist.train.images, y:mnist.train.labels, keep_prob:0.8})# 70% 工作
        test_acc = sess.run(arruracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})# 测试的时候所有的神经元都工作
        print("Iter" + str(epoch) + " Training Accuracy" + str(train_acc) + " Testing Accuracy" + str(test_acc))

Iter0 Training Accuracy0.90018183 Testing Accuracy0.935
Iter1 Training Accuracy0.9193818 Testing Accuracy0.9467
Iter2 Training Accuracy0.9247091 Testing Accuracy0.9562
Iter3 Training Accuracy0.9306727 Testing Accuracy0.9632
Iter4 Training Accuracy0.9328727 Testing Accuracy0.9633
Iter5 Training Accuracy0.9334 Testing Accuracy0.9673
Iter6 Training Accuracy0.9374545 Testing Accuracy0.9689
Iter7 Training Accuracy0.93854547 Testing Accuracy0.9691
Iter8 Training Accuracy0.93921816 Testing Accuracy0.9703
Iter9 Training Accuracy0.93954545 Testing Accuracy0.9708
Iter10 Training Accuracy0.9414 Testing Accuracy0.9727
Iter11 Training Accuracy0.9389455 Testing Accuracy0.9741
Iter12 Training Accuracy0.9422182 Testing Accuracy0.9742
Iter13 Training Accuracy0.9416909 Testing Accuracy0.9738
Iter14 Training Accuracy0.9410909 Testing Accuracy0.9747
Iter15 Training Accuracy0.94241816 Testing Accuracy0.9754
Iter16 Training Accuracy0.93954545 Testing Accuracy0.9755
Iter17 Training Accuracy0.9439273 Testing 

### 未使用dropout
Iter0 Training Accuracy0.9024182 Testing Accuracy0.907
Iter1 Training Accuracy0.92594546 Testing Accuracy0.9246
Iter2 Training Accuracy0.93814546 Testing Accuracy0.9375
...
Iter28 Training Accuracy0.98834544 Testing Accuracy0.971
Iter29 Training Accuracy0.98865455 Testing Accuracy0.9713
Iter30 Training Accuracy0.98883635 Testing Accuracy0.9726
存在过拟合情况

### 使用 dropout train = 0.8 number_neural=100
Iter0 Training Accuracy0.8772727 Testing Accuracy0.9032
Iter1 Training Accuracy0.8999091 Testing Accuracy0.9225
Iter2 Training Accuracy0.9084909 Testing Accuracy0.9342
...
Iter28 Training Accuracy0.9042182 Testing Accuracy0.9706
Iter29 Training Accuracy0.90752727 Testing Accuracy0.9702
Iter30 Training Accuracy0.90776366 Testing Accuracy0.9713